## Baseline variant 1
Optimize **w** (through the CRPS) regarding
* all **task 2 datasets** (2018-2021)
* **all countries**
* **all prediciton windows**
-> alle Monate rausnehmen, die nicht in allen Ländern sind -> gemeinsame Schnittmenge

for i in w's
    for i in dataset
        nbinomquant = ...
        for i in s vorhersagehorizont
            crps(w,s) = ...
        meancrps(dataset) = meancrps über alle s
    datensatz(w) = aktuelles w
    datensatz(w's meanCrps) = meanCrps über alle datasets

optimales w = min CRPS w
-> CRPS des optimalen w's


Problem: einige Länder haben << 36 Monate als Beobachtung
-> also unterscheiden in Länder 

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import os
from scipy.stats import nbinom
import CRPS.CRPS as pscore

# create the feature- and actuals-data list
# set the feature and actuals year lists
feature_years = ['2017','2018','2019','2020']
actual_years = ['2018','2019','2020','2021']

actuals_df_list = []
features_df_list = []

for i in range(len(feature_years)):
    # paths to the data
    absolute_path = os.path.abspath('')
    relative_path_features = "data\cm_features_to_oct" + feature_years[i] + ".parquet"
    relative_path_actuals = "data\cm_actuals_" + actual_years[i] + ".parquet"

    path_features = os.path.join(absolute_path, relative_path_features)
    path_actuals = os.path.join(absolute_path, relative_path_actuals)

    # append datasets to the lists
    actuals_df_list.append({'year':actual_years[i], 'data':pd.read_parquet(path_actuals, engine='pyarrow')})
    features_df_list.append({'year':feature_years[i], 'data':pd.read_parquet(path_features, engine='pyarrow')})


'2018'

In [86]:
baseline_predict_list = [[]]*len(feature_years)
window_list = list(range(2, 37))

# loop through datasets
for i in range(len(feature_years)):
    # sum of the conflict related deaths for each month
    monthly_totals_features = features_df_list[i]['data'].groupby('month_id').sum()
    monthly_totals_actuals = actuals_df_list[i]['data'].groupby('month_id').sum()

    baseline_predict_list[i] = {'actual_year':actual_years[i], 'prediction':[]}
    # loop through windows
    for j in range(len(window_list)):
        w = window_list[j] # current window

        baseline_predict_list[i]['prediction'].append({'window':w, 'quantile':[], 'fatalities':[]})

        # calculate n (r) and p via average/variance
        mean = pd.Series.mean(monthly_totals_features.tail(w).loc[:,'ged_sb'])
        var = pd.Series.var(monthly_totals_features.tail(w).loc[:,'ged_sb'])

        """ if w=1 mean=variance? -> Verschiebungssatz Var auch Null """

        n = (mean**2)/(var-mean) # equivalent to r
        p = mean/var

        dummy_fatalities_list = []
        dummy_quantile_list = []

        # compute all quantiles from 0.1% to 99.9%
        for k in range(10, 9999, 10):
            dummy_fatalities_list.append(nbinom.ppf(k/10000, n, p))
            dummy_quantile_list.append(str((k/10000)*100)+'%')

        baseline_predict_list[i]['prediction'][j]['quantile'] = dummy_quantile_list    
        baseline_predict_list[i]['prediction'][j]['fatalities'] = dummy_fatalities_list

        baseline_predict_list[i]['prediction'][j] = pd.DataFrame(baseline_predict_list[i]['prediction'][j])
        baseline_predict_list[i]['prediction'][j].set_index('quantile')

baseline_predict_list[1]['prediction']


[     window quantile  fatalities
 0         2     0.1%      3214.0
 1         2     0.2%      3261.0
 2         2     0.3%      3290.0
 3         2     0.4%      3312.0
 4         2     0.5%      3329.0
 ..      ...      ...         ...
 994       2    99.5%      4637.0
 995       2    99.6%      4658.0
 996       2    99.7%      4686.0
 997       2    99.8%      4723.0
 998       2    99.9%      4783.0
 
 [999 rows x 3 columns],
      window quantile  fatalities
 0         3     0.1%      3430.0
 1         3     0.2%      3465.0
 2         3     0.3%      3487.0
 3         3     0.4%      3503.0
 4         3     0.5%      3516.0
 ..      ...      ...         ...
 994       3    99.5%      4449.0
 995       3    99.6%      4464.0
 996       3    99.7%      4483.0
 997       3    99.8%      4509.0
 998       3    99.9%      4550.0
 
 [999 rows x 3 columns],
      window quantile  fatalities
 0         4     0.1%      2009.0
 1         4     0.2%      2133.0
 2         4     0.3%      2

In [90]:
baseline_predict_list[0]['prediction'][20]

,window,quantile,fatalities
0,22,0.1%,1035.0
1,22,0.2%,1213.0
2,22,0.3%,1333.0
3,22,0.4%,1426.0
4,22,0.5%,1505.0
...,...,...,...
994,22,99.5%,17300.0
995,22,99.6%,17727.0
996,22,99.7%,18273.0
997,22,99.8%,19032.0


In [38]:
# Gruppiere den Datensatz nach 'Land ID' und zähle die Anzahl der Monate für jedes Land
land_counts = df_features.groupby('country_id').size()

# Finde den minimalen und maximalen Monat über alle Länder
min_month = df_features.index.get_level_values('month_id').min()
max_month = df_features.index.get_level_values('month_id').max()

""" # Iteriere über die Länder und markiere fehlende Monate zum Löschen
to_delete = []
for land, count in land_counts.items():
    if count != max_month - min_month + 1:
        missing_months = set(range(min_month, max_month+1)) - set(df_features.loc[land].index.get_level_values('month_id'))
        to_delete.extend([(land, month) for month in missing_months])

# Filtere den Datensatz, um nur die Monate beizubehalten, die nicht zum Löschen markiert wurden
data_filtered = df_features.drop(to_delete)
data_filtered """
land_df = pd.DataFrame(land_counts)
land_df[land_df.iloc[:,0] < 36]

,0
country_id,
185,10
186,10
188,23
189,20
192,3
196,5
197,5
230,21
247,6


## Baseline variant 2
Optimize **w** (through the CRPS) regarding
* all **task 2 datasets** (2018-2021)
* **inidividual countries**
* **all prediciton windows**

for country x
    for i in w's
        for i in dataset
            nbinomquant = ...
            for i in s vorhersagehorizont
                crps(x,w,s) = ...
            meancrps(x,dataset) = meancrps über alle s
        datensatz(x,w) = aktuelles w
        datensatz(x,w's meanCrps) = meanCrps über alle datasets des landes x
    datensatz2(x,opimales w) = min CRPS w
    datensatz2(x, w's meanCrps) = optimaler Crps des landes x unter opti. w

-> CRPS = mittelrwert der CRPS über aller länder

## Baseline variant 3
Optimize **w** (through the CRPS) regarding
* all **task 2 datasets** (2018-2021)
* **all countries**
* **inidvidual prediciton windows**
-> alle Monate rausnehmen, die nicht in allen Ländern sind -> gemeinsame Schnittmenge

for i in s (prediction window)
    for i in w's
        for i in dataset
            nbinomquant = ...
                crps(w,s) = ...
                aktuelles w speichern
    mincrps(s) = kleinster crps über alle w
    w = zugehöriges w des mincrps
    s
CRPS = mean über alle mincrps

optimales w = min CRPS w
-> CRPS des optimalen w's